In [32]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, GRU, LSTM
import numpy as np

In [33]:
num_words = 10000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=num_words)

word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

decoded_train_data = [' '.join([reverse_word_index.get(i - 3, '?') for i in sequence]) for sequence in train_data]
decoded_test_data = [' '.join([reverse_word_index.get(i - 3, '?') for i in sequence]) for sequence in test_data]

tokenizer = Tokenizer(num_words=num_words)
all_data = decoded_train_data + decoded_test_data
tokenizer.fit_on_texts(all_data)

train_sequences = tokenizer.texts_to_sequences(decoded_train_data)
test_sequences = tokenizer.texts_to_sequences(decoded_test_data)

maxlen = 100
train_data = pad_sequences(train_sequences, maxlen=maxlen)
test_data = pad_sequences(test_sequences, maxlen=maxlen)

In [34]:
#Проста рекурентна нейронна мережа (SimpleRNN)
model_rnn = Sequential()
model_rnn.add(Embedding(num_words, 32))
model_rnn.add(SimpleRNN(32))
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_rnn.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, None, 32)          320000    
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 322113 (1.23 MB)
Trainable params: 322113 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
#Довга краткочасна пам'ять (LSTM)
model_lstm = Sequential()
model_lstm.add(Embedding(num_words, 32))
model_lstm.add(LSTM(32))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_lstm.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, None, 32)          320000    
                                                                 
 lstm_4 (LSTM)               (None, 32)                8320      
                                                                 
 dense_12 (Dense)            (None, 1)                 33        
                                                                 
Total params: 328353 (1.25 MB)
Trainable params: 328353 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
#Gated Recurrent Unit (GRU)
model_gru = Sequential()
model_gru.add(Embedding(num_words, 32))
model_gru.add(GRU(32))
model_gru.add(Dense(1, activation='sigmoid'))
model_gru.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_gru.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, None, 32)          320000    
                                                                 
 gru_4 (GRU)                 (None, 32)                6336      
                                                                 
 dense_13 (Dense)            (None, 1)                 33        
                                                                 
Total params: 326369 (1.24 MB)
Trainable params: 326369 (1.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
#Комбінація різних типів шарів
model_combined = Sequential()
model_combined.add(Embedding(num_words, 32))
model_combined.add(LSTM(64, return_sequences=True))
model_combined.add(GRU(32))
model_combined.add(Dense(1, activation='sigmoid'))
model_combined.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_combined.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, None, 32)          320000    
                                                                 
 lstm_5 (LSTM)               (None, None, 64)          24832     
                                                                 
 gru_5 (GRU)                 (None, 32)                9408      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 354273 (1.35 MB)
Trainable params: 354273 (1.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
history_rnn = model_rnn.fit(train_data, train_labels, epochs=50, batch_size=160, validation_split=0.2)

Epoch 1/50
125/125 [==============================] - 9s 49ms/step - loss: 0.6846 - accuracy: 0.5505 - val_loss: 0.6663 - val_accuracy: 0.6110
Epoch 2/50
125/125 [==============================] - 6s 44ms/step - loss: 0.4737 - accuracy: 0.7891 - val_loss: 0.4008 - val_accuracy: 0.8270
Epoch 3/50
125/125 [==============================] - 5s 43ms/step - loss: 0.2314 - accuracy: 0.9139 - val_loss: 0.4184 - val_accuracy: 0.8138
Epoch 4/50
125/125 [==============================] - 5s 43ms/step - loss: 0.0980 - accuracy: 0.9714 - val_loss: 0.4529 - val_accuracy: 0.8260
Epoch 5/50
125/125 [==============================] - 5s 42ms/step - loss: 0.0381 - accuracy: 0.9934 - val_loss: 0.5085 - val_accuracy: 0.8290
Epoch 6/50
125/125 [==============================] - 5s 43ms/step - loss: 0.0146 - accuracy: 0.9985 - val_loss: 0.5678 - val_accuracy: 0.8148
Epoch 7/50
125/125 [==============================] - 5s 42ms/step - loss: 0.0077 - accuracy: 0.9995 - val_loss: 0.6371 - val_accuracy: 0.8242

In [39]:
history_lstm = model_lstm.fit(train_data, train_labels, epochs=50, batch_size=160, validation_split=0.2)

Epoch 1/50
125/125 [==============================] - 20s 119ms/step - loss: 0.5214 - accuracy: 0.7244 - val_loss: 0.3820 - val_accuracy: 0.8372
Epoch 2/50
125/125 [==============================] - 14s 110ms/step - loss: 0.2788 - accuracy: 0.8873 - val_loss: 0.3378 - val_accuracy: 0.8540
Epoch 3/50
125/125 [==============================] - 14s 110ms/step - loss: 0.2068 - accuracy: 0.9243 - val_loss: 0.3785 - val_accuracy: 0.8428
Epoch 4/50
125/125 [==============================] - 14s 109ms/step - loss: 0.1618 - accuracy: 0.9444 - val_loss: 0.4066 - val_accuracy: 0.8444
Epoch 5/50
125/125 [==============================] - 14s 109ms/step - loss: 0.1276 - accuracy: 0.9578 - val_loss: 0.4302 - val_accuracy: 0.8434
Epoch 6/50
125/125 [==============================] - 14s 110ms/step - loss: 0.1025 - accuracy: 0.9674 - val_loss: 0.5079 - val_accuracy: 0.8354
Epoch 7/50
125/125 [==============================] - 14s 110ms/step - loss: 0.0811 - accuracy: 0.9748 - val_loss: 0.5688 - val_ac

In [40]:
history_gru = model_gru.fit(train_data, train_labels, epochs=50, batch_size=160, validation_split=0.2)

Epoch 1/50
125/125 [==============================] - 18s 106ms/step - loss: 0.5456 - accuracy: 0.7053 - val_loss: 0.3729 - val_accuracy: 0.8410
Epoch 2/50
125/125 [==============================] - 12s 96ms/step - loss: 0.2913 - accuracy: 0.8812 - val_loss: 0.3496 - val_accuracy: 0.8564
Epoch 3/50
125/125 [==============================] - 12s 97ms/step - loss: 0.2191 - accuracy: 0.9165 - val_loss: 0.3614 - val_accuracy: 0.8458
Epoch 4/50
125/125 [==============================] - 11s 90ms/step - loss: 0.1777 - accuracy: 0.9359 - val_loss: 0.4238 - val_accuracy: 0.8440
Epoch 5/50
125/125 [==============================] - 12s 93ms/step - loss: 0.1488 - accuracy: 0.9483 - val_loss: 0.4538 - val_accuracy: 0.8408
Epoch 6/50
125/125 [==============================] - 12s 98ms/step - loss: 0.1240 - accuracy: 0.9590 - val_loss: 0.4566 - val_accuracy: 0.8352
Epoch 7/50
125/125 [==============================] - 12s 94ms/step - loss: 0.1076 - accuracy: 0.9654 - val_loss: 0.5261 - val_accuracy

In [41]:
history_combined = model_combined.fit(train_data, train_labels, epochs=50, batch_size=160, validation_split=0.2)

Epoch 1/50
125/125 [==============================] - 47s 318ms/step - loss: 0.4730 - accuracy: 0.7613 - val_loss: 0.3449 - val_accuracy: 0.8478
Epoch 2/50
125/125 [==============================] - 37s 297ms/step - loss: 0.2598 - accuracy: 0.8960 - val_loss: 0.3444 - val_accuracy: 0.8516
Epoch 3/50
125/125 [==============================] - 37s 299ms/step - loss: 0.1973 - accuracy: 0.9253 - val_loss: 0.3862 - val_accuracy: 0.8450
Epoch 4/50
125/125 [==============================] - 37s 300ms/step - loss: 0.1677 - accuracy: 0.9395 - val_loss: 0.3923 - val_accuracy: 0.8378
Epoch 5/50
125/125 [==============================] - 37s 299ms/step - loss: 0.1330 - accuracy: 0.9542 - val_loss: 0.4614 - val_accuracy: 0.8366
Epoch 6/50
125/125 [==============================] - 37s 299ms/step - loss: 0.0989 - accuracy: 0.9673 - val_loss: 0.5815 - val_accuracy: 0.8270
Epoch 7/50
125/125 [==============================] - 37s 299ms/step - loss: 0.0891 - accuracy: 0.9706 - val_loss: 0.6284 - val_ac

In [42]:
test_loss_rnn, test_accuracy_rnn = model_rnn.evaluate(test_data, test_labels)
print(f'Точність алгоритму RNN: {np.round(test_accuracy_rnn*100, 2)}%')

test_loss_lstm, test_accuracy_lstm = model_lstm.evaluate(test_data, test_labels)
print(f'Точність алгоритму LSTM: {np.round(test_accuracy_lstm*100, 2)}%')

test_loss_gru, test_accuracy_gru = model_gru.evaluate(test_data, test_labels)
print(f'Точність алгоритму GRU: {np.round(test_accuracy_gru*100, 2)}%')

test_loss_combined, test_accuracy_combined = model_combined.evaluate(test_data, test_labels)
print(f'Точність комбінованого алгоритму: {np.round(test_accuracy_combined*100, 2)}%')

782/782 [==============================] - 3s 3ms/step - loss: 1.0984 - accuracy: 0.8093
Точність алгоритму RNN: 80.93%
782/782 [==============================] - 6s 8ms/step - loss: 1.4192 - accuracy: 0.8109
Точність алгоритму LSTM: 81.09%
782/782 [==============================] - 9s 12ms/step - loss: 1.7648 - accuracy: 0.7968
Точність алгоритму GRU: 79.68%
782/782 [==============================] - 19s 24ms/step - loss: 1.5116 - accuracy: 0.8123
Точність комбінованого алгоритму: 81.23%
